# Notebook destinado a fazer teste de leituras de CSV
## PJ e Socios e geração de graphframe
### Exemplifica uso do módulto trata_dados.py

In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from graphframes import *
from pyspark.sql.functions import col, from_json, substring, when, length, split, lit

In [2]:
from trata_dados import trata_empresa
from trata_dados import trata_socio
from trata_dados import monta_grafo

In [3]:
# Dados de carga
csv_pj = 'dataset/completo/empresas_estabelecimentos*.csv'
csv_socios = 'dataset/completo/socios*.csv'

In [4]:
spark = SparkSession.builder \
        .config("spark.driver.memory", "128g") \
        .appName("trata_dados_3") \
        .getOrCreate()

sc = spark.sparkContext
sc.setCheckpointDir('graphframes_cps')

24/08/21 17:34:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
empresas = trata_empresa(csv_pj, spark)
# numero_de_linhas = pj.count()
# print(f"O DataFrame contém {numero_de_linhas} linhas.")

In [6]:
socios = trata_socio(csv_socios, spark)
# numero_de_linhas = socio.count()
# print(f"O DataFrame contém {numero_de_linhas} linhas.")

In [6]:
# Cria df para vértices do tipo Pessoa Física:
# pf = socio.filter(col("te_dados_sc.identificadorSocio") == '2').select(col("dst").alias("id")).distinct()

# cols_pj = [col_name for col_name in pj.columns if col_name != 'id']
# for col_name in cols_pj:
#    pf = pf.withColumn(col_name, lit(None))

# cria tipo de pessoa na nos vértices
# pj = pj.withColumn("id_tipoPessoa", lit(1))
# pf = pf.withColumn("id_tipoPessoa", lit(2))

# vertex = pj.union(pf).orderBy("id")

# verificando unidade de chaves:
# count_df = vertex.groupBy('id').count()
# repeated_vertex = count_df.filter(col('count') > 1)
# repeated_vertex.show()

In [7]:
# Criando o grafo de relacionamento empresas->socios
g = monta_grafo(empresas, socios)
# result=g.connectedComponents()

In [8]:
result=g.connectedComponents()

In [9]:
# conectados = result.select("id", "component", "id_tipoPessoa").orderBy("component")
# conectados.show()
component_sizes = result.groupBy('component').count()
largest_component = component_sizes.orderBy(col('count').desc()).first()
largest_component_id = largest_component['component']
largest_component_vertices = result.filter(result['component'] == largest_component_id)
largest_component_vertices.show()

24/08/21 18:09:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+--------------------+------------------+--------------------+-------------+---------+
|            id|         te_dados_em|id_estabelecimento|         te_dados_es|id_tipoPessoa|component|
+--------------+--------------------+------------------+--------------------+-------------+---------+
|00009350518643|                NULL|              NULL|                NULL|            2|    84686|
|00009502044800|                NULL|              NULL|                NULL|            2|    84686|
|      41298981|{03, 000000002000...|            000182|{SP, 04042005, 11...|            1|    84686|
|00085867701590|                NULL|              NULL|                NULL|            2|    84686|
|00002269877101|                NULL|              NULL|                NULL|            2|    84686|
|00096651610244|                NULL|              NULL|                NULL|            2|    84686|
|00033332486896|                NULL|              NULL|                NULL|     

In [10]:
edges = g.edges.filter("src='03235791' or src='03410522' or src='04038317' or src='00002117401872' or src='04100516'")
edges.show()

+--------+--------------+--------------------+
|     src|           dst|         te_dados_sc|
+--------+--------------+--------------------+
|03235791|00000610508067|{000, 20170822, ,...|
|03235791|00001683293797|{000, 20230102, ,...|
|03235791|00002117401872|{000, 20021018, ,...|
|03235791|00003604616549|{000, 20021018, ,...|
|03235791|00009101021710|{000, 20230102, ,...|
|03235791|00009442262807|{000, 20230102, ,...|
|03235791|00029435367844|{000, 20230102, ,...|
|03235791|      73178600|{000, 19990624, ,...|
|03410522|00000610508067|{000, 20180611, ,...|
|03410522|00002117401872|{000, 20021029, ,...|
|03410522|00003604616549|{000, 20021029, ,...|
|03410522|00009101021710|{000, 20200428, ,...|
|03410522|00009442262807|{000, 20240321, ,...|
|03410522|00010793641705|{000, 20200428, ,...|
|03410522|      73178600|{000, 20061201, ,...|
|04038317|00002117401872|{000, 20080626, ,...|
|04038317|00003604616549|{000, 20080626, ,...|
|04038317|00016236648883|{000, 20100920, ,...|
|04038317|   

In [11]:
vertex = g.vertices.filter("id = '00002117401872' or id ='00001683293797'")
vertex.show()

+--------------+-----------+------------------+-----------+-------------+
|            id|te_dados_em|id_estabelecimento|te_dados_es|id_tipoPessoa|
+--------------+-----------+------------------+-----------+-------------+
|00001683293797|       NULL|              NULL|       NULL|            2|
|00002117401872|       NULL|              NULL|       NULL|            2|
+--------------+-----------+------------------+-----------+-------------+



In [12]:
spark.stop()